## LSSTCam Astrometry learning with GAIA

Craig Lage - 19-May-25

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from lsst.daf.butler import Butler
import lsst.afw.geom as afwGeom
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from astroquery.gaia import Gaia
from lsst.geom import SpherePoint,Angle,Extent2I,Box2I,Extent2D,Point2D, Point2I
from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms
from lsst.obs.lsst import LsstCam
from lsst.summit.utils.plotting import plot

In [ ]:
def MakeShiftedWCSScience(camera, ref_detector, ref_wcs, img_detector):
    img_bbox = img_detector.getBBox()
    ref_bbox = ref_detector.getBBox()
    ref_detName = ref_detector.getName()
    img_detName = img_detector.getName()
    ref_lct = LsstCameraTransforms(camera, ref_detName)
    refX, refY = ref_lct.ccdPixelToFocalMm(ref_bbox.centerX, ref_bbox.centerY, ref_detName)
    img_lct = LsstCameraTransforms(camera, img_detName)
    imgX, imgY = img_lct.ccdPixelToFocalMm(img_bbox.centerX, img_bbox.centerY, img_detName)
    # The factor of 100 below is 1000 microns/mm / 10 microns/pixel = 100 pixels/mm.
    fpX = (imgX - refX) * 100 + img_bbox.centerX # Convert to pixels and recenter
    fpY = (imgY - refY) * 100 + img_bbox.centerY # Convert to pixels and recenter
    cdMatrix = ref_wcs.getCdMatrix()
    # now rotate the CD matrix into the right orientation.
    nRot = img_detector.getOrientation().getNQuarter()
    rot = ((90.0 * nRot) % 360.0) * np.pi / 180.0
    rot_matrix = np.array([[np.cos(rot), np.sin(rot)],
                                [-np.sin(rot), np.cos(rot)]])
    cdMatrix_rot = np.dot(rot_matrix, cdMatrix)
    pixelOrigin = Point2D(img_bbox.centerX, img_bbox.centerY)
    skyOrigin = ref_wcs.pixelToSky(Point2D(fpX, fpY))
    img_wcs = afwGeom.makeSkyWcs(pixelOrigin, skyOrigin, cdMatrix_rot)
    return img_wcs


In [ ]:
def MakeShiftedWCSGuider(camera, ref_detector, ref_wcs, img_detector, expId):
    ref_bbox = ref_detector.getBBox()
    img_bbox = img_detector.getBBox()
    ref_detName = ref_detector.getName()
    img_detName = img_detector.getName()
    ref_lct = LsstCameraTransforms(camera, ref_detName)
    refX, refY = ref_lct.ccdPixelToFocalMm(ref_bbox.centerX, ref_bbox.centerY, ref_detName)
    img_lct = LsstCameraTransforms(camera, img_detName)
    raw = butler.get("guider_raw", exposure=expId, detector=img_detector.getId(), instrument='LSSTCam')
    mdata = raw.metadata
    cols = mdata['ROICOLS']
    rows = mdata['ROIROWS']
    llX = mdata['ROICOL']
    llY = mdata['ROIROW']
    urX = llX + mdata['ROICOLS']
    urY = llY + mdata['ROIROWS']
    stamp_bbox = Box2I(Point2I(0, 0), Point2I(cols, rows))
    stamp_ampName = ampDict[mdata['ROISEG']]
    x0, y0 = img_lct.ampPixelToCcdPixel(llX, llY, stamp_ampName, img_detName)
    x1, y1 = img_lct.ampPixelToCcdPixel(urX, urY, stamp_ampName, img_detName)
    x0 = int(x0)
    y0 = int(y0)
    x1 = int(x1)
    y1 = int(y1)
    if x1 < x0:
        x0, x1 = x1, x0
    if y1 < y0:
        y0, y1 = y1, y0
    imgX, imgY = img_lct.ccdPixelToFocalMm(img_bbox.centerX, img_bbox.centerY, img_detName)
    # The factor of 100 below is 1000 microns/mm / 10 microns/pixel = 100 pixels/mm.
    fpX = (imgX - refX) * 100 + img_bbox.centerX # Convert to pixels and recenter
    fpY = (imgY - refY) * 100 + img_bbox.centerY # Convert to pixels and recenter
    cdMatrix = ref_wcs.getCdMatrix()
    # now rotate the CD matrix into the right orientation.
    nRot = img_detector.getOrientation().getNQuarter()
    rot = ((90.0 * nRot) % 360.0) * np.pi / 180.0
    rot_matrix = np.array([[np.cos(rot), np.sin(rot)],
                                [-np.sin(rot), np.cos(rot)]])
    cdMatrix_rot = np.dot(rot_matrix, cdMatrix)
    pixelOrigin = Point2D(img_bbox.centerX, img_bbox.centerY)
    skyOrigin = ref_wcs.pixelToSky(Point2D(fpX, fpY))
    img_wcs = afwGeom.makeSkyWcs(pixelOrigin, skyOrigin, cdMatrix_rot)
    # Now shift to put the LL corner of the stamp as the origin
    stamp_wcs = img_wcs.copyAtShiftedPixelOrigin(Extent2D(-x0, -y0))
    return stamp_wcs, stamp_bbox


In [ ]:
def QueryGAIA(wcs, bbox, ra_center, dec_center, magLimit=17.0):
    radius = max(bbox.getDimensions()) / 2.0 * np.sqrt(2.0) * 0.2
    radius = radius * u.arcsec # Enough to cover the bbox    
    cols = ['ra', 'dec', 'phot_g_mean_mag', 'designation']
    # Construct the ADQL query
    query = f"""
    SELECT *
    FROM gaiadr3.gaia_source
    WHERE CONTAINS(POINT({ra_center}, {dec_center}), CIRCLE(ra, dec, {radius.to(u.degree).value})) = 1
    AND phot_g_mean_mag < {magLimit} 
    """
    # Launch the query
    job = Gaia.launch_job_async(query)
    results = job.get_results()
    gaia_table = results.to_pandas()
    gaia_table = gaia_table[cols]
    # using the wcs to locate the stars inside the CCD minus npixedge
    ccdx,ccdy = wcs.skyToPixelArray(gaia_table['ra'],gaia_table['dec'], degrees=True)
    inCCD = bbox.contains(ccdx,ccdy)
    gaia_table['ccdx'] = ccdx
    gaia_table['ccdy'] = ccdy
    gaia_table['inCCD'] = inCCD
    return gaia_table


In [ ]:
def CheckWCSScience(camera, wcs, expId, detector, magLimit):
    detName = detector.getName()
    bbox = detector.getBBox()
    calexp = butler.get('preliminary_visit_image', detector=detector.getId(), visit=expId)
    skyCenter = wcs.pixelToSky(Point2D(bbox.centerX, bbox.centerY))
    ra_center = skyCenter.getRa().asDegrees()
    dec_center = skyCenter.getDec().asDegrees()
    gaia_table = QueryGAIA(wcs, bbox, ra_center, dec_center, magLimit=magLimit)
    xs = []
    ys = []
    mags = []
    for i in range(len(gaia_table)):
        x = gaia_table['ccdx'][i]
        y = gaia_table['ccdy'][i]
        if gaia_table['inCCD'][i]:
            xs.append(x)
            ys.append(y)
            mags.append(f"{gaia_table['phot_g_mean_mag'][i]:.1f}")
    
    fig = plt.figure(figsize=(16,16))
    
    myPlot = plot(calexp, stretch='ccs', figure=fig)
    ax = myPlot.get_axes()[0]
    ax.scatter(xs, ys\
                ,facecolors='none', edgecolors='magenta', s=200, lw=2)
    ax.set_title(f"{expId} detector {detector.getId()}")
    return fig

In [ ]:
# The line below needs to be checked when we start using other amps.
ampDict = {'Segment00':'C00', 'Segment01':'C01', 'Segment02':'C02', 'Segment03':'C03', 
           'Segment04':'C04', 'Segment05':'C05', 'Segment06':'C06', 'Segment07':'C07', 
           'Segment08':'C17', 'Segment09':'C16', 'Segment10':'C15', 'Segment11':'C14', 
           'Segment12':'C13', 'Segment13':'C12', 'Segment14':'C11', 'Segment15':'C10', }


def CheckWCSGuider(camera, wcs, bbox, expId, detector, magLimit):
    detName = detector.getName()
    if expId > 2025050900000:
        detName = detector.getName()
    else:
        print("Swapping SG0, SG1")
        # This is necessary because SG0 and SG1 were swapped for a time
        if detName.split('_')[1] == 'SG0':
            detName = detName.split('_')[0] + '_SG1'
        else:
            detName = detName.split('_')[0] + '_SG0'
    detector = camera[detName]
    raw = butler.get("guider_raw", exposure=expId, detector=detector.getId(), instrument='LSSTCam')
    arr = raw[2].stamp_im.image.array
    arr = arr - np.median(arr)
    arr = np.fliplr(arr)
    skyCenter = wcs.pixelToSky(Point2D(bbox.centerX, bbox.centerY))
    ra_center = skyCenter.getRa().asDegrees()
    dec_center = skyCenter.getDec().asDegrees()
    gaia_table = QueryGAIA(wcs, bbox, ra_center, dec_center, magLimit=magLimit)
    xs = []
    ys = []
    mags = []

    for i in range(len(gaia_table)):
        x = gaia_table['ccdx'][i]
        y = gaia_table['ccdy'][i]
        if gaia_table['inCCD'][i]:
            xs.append(x)
            ys.append(y)
            mags.append(f"{gaia_table['phot_g_mean_mag'][i]:.1f}")

    fig = plt.figure(figsize=(8, 12))
    
    myPlot = plot(arr, stretch='ccs', figure=fig)
    ax = myPlot.get_axes()[0]
    ax.scatter(xs, ys\
                ,facecolors='none', edgecolors='magenta', s=2000, lw=4)
    ax.set_title(f"{expId} detector {detName}")
    return fig, gaia_table

In [ ]:
butler = Butler('LSSTCam', collections=["LSSTCam/raw/all", "LSSTCam/calib", 
                                        "LSSTCam/runs/quickLook", "LSSTCam/raw/guider"])
instrument = 'LSSTCam'

# Check the WCS on the central CCD

In [ ]:
expId = 2025071800250

ref_detName = 'R22_S11'
camera = LsstCam.getCamera()
ref_detector = camera[ref_detName]
calexp = butler.get('preliminary_visit_image', detector=ref_detector.getId(), visit=expId)
ref_wcs = calexp.getWcs()
magLimit = 15.0

fig = CheckWCSScience(camera, ref_wcs, expId, ref_detector, magLimit)

# Check extrapolating to another science CCD

In [ ]:
expId = 2025071800250

ref_detName = 'R22_S11'
camera = LsstCam.getCamera()
ref_detector = camera[ref_detName]
calexp = butler.get('preliminary_visit_image', detector=ref_detector.getId(), visit=expId)
ref_wcs = calexp.getWcs()
magLimit = 15.0

img_detName = 'R33_S22'
img_detector = camera[img_detName]
img_wcs = MakeShiftedWCSScience(camera, ref_detector, ref_wcs, img_detector)
fig = CheckWCSScience(camera, img_wcs, expId, img_detector, magLimit)

# Extrapolate to a guider CCD and check just one stamp

In [ ]:
expId = 2025071800250

ref_detName = 'R22_S11'
camera = LsstCam.getCamera()
ref_detector = camera[ref_detName]
calexp = butler.get('preliminary_visit_image', detector=ref_detector.getId(), visit=expId)
ref_wcs = calexp.getWcs()

dets = ['R00_SG0', 'R00_SG1', 'R04_SG0', 'R04_SG1', 'R40_SG0', 'R40_SG1', 'R44_SG0', 'R44_SG1']
img_detName = dets[6]
img_detector = camera[img_detName]
stamp_wcs, stamp_bbox = MakeShiftedWCSGuider(camera, ref_detector, ref_wcs, img_detector, expId)
fig, gaia_table = CheckWCSGuider(camera, stamp_wcs, stamp_bbox, expId, img_detector, magLimit=16.5)
ax = fig.get_axes()[0]
gaia_table = gaia_table.sort_values(by='phot_g_mean_mag', ascending=True)
gaia_table = gaia_table.reset_index(drop=True)
numGaia = min(8, len(gaia_table))
for n in range(numGaia):
    if gaia_table['inCCD'][n]:
        text = f"{gaia_table['designation'][n]}  {gaia_table['ccdx'][n]:.1f}  {gaia_table['ccdy'][n]:.1f}  {gaia_table['phot_g_mean_mag'][n]:.2f}"
        ax.text(0.1, 0.2 - 0.02 * n, text, transform=fig.transFigure, fontsize=12)

# Make a PDF with all 8 stamps

In [ ]:
expId = 2025071800250
pdf = PdfPages(f"/home/cslage/DATA/Guider_Stamps_{expId}.pdf")
ref_detName = 'R22_S11'
camera = LsstCam.getCamera()
ref_detector = camera[ref_detName]
calexp = butler.get('preliminary_visit_image', detector=ref_detector.getId(), visit=expId)
ref_wcs = calexp.getWcs()

dets = ['R00_SG0', 'R00_SG1', 'R04_SG0', 'R04_SG1', 'R40_SG0', 'R40_SG1', 'R44_SG0', 'R44_SG1']

for i in range(8):
    img_detName = dets[i]
    img_detector = camera[img_detName]
    stamp_wcs, stamp_bbox = MakeShiftedWCSGuider(camera, ref_detector, ref_wcs, img_detector, expId)
    fig, gaia_table = CheckWCSGuider(camera, stamp_wcs, stamp_bbox, expId, img_detector, magLimit=16.5)
    ax = fig.get_axes()[0]
    gaia_table = gaia_table.sort_values(by='phot_g_mean_mag', ascending=True)
    gaia_table = gaia_table.reset_index(drop=True)
    numGaia = min(8, len(gaia_table))
    counter = 0
    for n in range(len(gaia_table)):
        if gaia_table['inCCD'][n]:
            text = f"{gaia_table['designation'][n]}  {gaia_table['ccdx'][n]:.1f}  {gaia_table['ccdy'][n]:.1f}  {gaia_table['phot_g_mean_mag'][n]:.2f}"
            ax.text(0.1, 0.2 - 0.02 * counter, text, transform=fig.transFigure, fontsize=10)
            counter += 1
        if counter > numGaia:
            break
    pdf.savefig(fig)
    fig.clf()
pdf.close()
